패키지 로드

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import urllib.request
import json

SSL 관련 함수 설정

In [ ]:
import requests
import urllib3
import ssl


class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

페이지 수 계산 함수

In [3]:
def page_count(product_num):
    if product_num % 20 == 0:
        page = int(product_num / 20)
    else:
        page = int(product_num/20) + 1
    return page

beautifulsoup url 읽기 함수

In [5]:
def url_soup(url):
    res = get_legacy_session().get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    soup = json.loads(soup.decode('utf-8'))
    return soup

제품 리뷰 수집 함수

In [4]:

def review_product(id, reviewCount):
    review_list_temp = []
    review_temp = []
    page_review = page_count(reviewCount)
    for page in range(page_review):
        url_review = f'https://gift.kakao.com/a/v2/review/products/{id}?page={page}&sortProperty=SCORE&size=20&_=1692145409357'
        soup_dict = url_soup(url_review)
        for n in range(20):
            try:
                review_text = soup_dict['reviewList']['contents'][n]['content']
                rating = soup_dict['reviewList']['contents'][n]['product']['rating']
                review_tag1 = soup_dict['reviewList']['contents'][n]['tag1']
                review_tag2 = soup_dict['reviewList']['contents'][n]['tag2']
                review_time = soup_dict['reviewList']['contents'][n]['displayStateInfo']['displayedAt']
                review_temp = product_details + [review_text, rating, review_tag1, review_tag2, review_time]
                review_list_temp.append(review_temp)
            except:continue
    return review_list_temp

데이터 수집

In [6]:
review_list = []
url = f'https://gift.kakao.com/a/v2/search/products?query=%EA%BF%80%EC%8A%A4%ED%8B%B1&categoryCode=102104&categoryName=%EC%8C%80%2F%EA%B3%BC%EC%9D%BC%2F%EB%86%8D%EC%B6%95%EC%88%98%EC%82%B0%EB%AC%BC&sortProperty=REVIEW_COUNT&sortDir=desc&page=0&_=1692150737323'
soup_product_dict = url_soup(url)
product_num = soup_product_dict['products']['totalCount']
page_product = page_count(product_num)
for page in range(page_product):
    url_products = f'https://gift.kakao.com/a/v2/search/products?query=%EA%BF%80%EC%8A%A4%ED%8B%B1&categoryCode=102104&categoryName=%EC%8C%80%2F%EA%B3%BC%EC%9D%BC%2F%EB%86%8D%EC%B6%95%EC%88%98%EC%82%B0%EB%AC%BC&sortProperty=REVIEW_COUNT&sortDir=desc&page={page}&_=1692150737323'
    soup_product_dict = url_soup(url_products)
    for product_n in range(20):
        try:
            if ('꿀' in soup_product_dict['products']['contents'][product_n]['name']) & ('스틱' in soup_product_dict['products']['contents'][product_n]['name']):
                id = soup_product_dict['products']['contents'][product_n]['id']
                name = soup_product_dict['products']['contents'][product_n]['name']
                brand = soup_product_dict['products']['contents'][product_n]['brand']['name']
                sellingPrice = soup_product_dict['products']['contents'][product_n]['price']['sellingPrice']
                wishCount = soup_product_dict['products']['contents'][product_n]['wish']['wishCount']
                reviewCount = reviewCount = soup_product_dict['products']['contents'][product_n]['review']['reviewCount']
                tag_1 = soup_product_dict['products']['contents'][product_n]['tierTag']['tierTags'][0]['tags']
                tag_2 = soup_product_dict['products']['contents'][product_n]['tierTag']['tierTags'][1]['tags']
                product_details = [id, name, brand, sellingPrice, wishCount, reviewCount, tag_1, tag_2]
                review_list = review_list + review_product(id, reviewCount)
                print((page*20)+product_n+1, '/', product_num, '완료', '-'*20)
            else:
                print((page*20)+product_n+1, '/', product_num, '완료', '-'*20)
        except:
            continue

1 / 230 완료 --------------------
2 / 230 완료 --------------------
3 / 230 완료 --------------------
6 / 230 완료 --------------------
7 / 230 완료 --------------------
8 / 230 완료 --------------------
10 / 230 완료 --------------------
11 / 230 완료 --------------------
13 / 230 완료 --------------------
14 / 230 완료 --------------------
15 / 230 완료 --------------------
16 / 230 완료 --------------------
17 / 230 완료 --------------------
18 / 230 완료 --------------------
19 / 230 완료 --------------------
20 / 230 완료 --------------------
21 / 230 완료 --------------------
22 / 230 완료 --------------------
23 / 230 완료 --------------------
24 / 230 완료 --------------------
25 / 230 완료 --------------------
26 / 230 완료 --------------------
27 / 230 완료 --------------------
28 / 230 완료 --------------------
29 / 230 완료 --------------------
30 / 230 완료 --------------------
31 / 230 완료 --------------------
32 / 230 완료 --------------------
33 / 230 완료 --------------------
34 / 230 완료 --------------------
35 / 230 완료 ----

In [8]:
review_df = pd.DataFrame(review_list, columns = ['id', 'name', 'brand', 'sellingPrice', 'wishCount', 'reviewCount', 'product_tag1', 'product_tag2',
                                                 'review_text', 'rating', 'review_tag1', 'review_tag2', 'review_time'])

데이터 저장

In [26]:
review_df.to_csv(r"C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\kakao_data.csv")

데이터 확인

In [28]:
review_df['id'].unique()

48

In [29]:
len(review_df)

1922

SSL 관련 함수 예시

In [ ]:
res = get_legacy_session().get('https://gift.kakao.com/a/v2/review/products/2270784?page=0&sortProperty=SCORE&size=20&_=1692145409357')
soup = BeautifulSoup(res.text, 'html.parser')
soup_dict = json.loads(soup.decode('utf-8'))